In this iPython notebook, I looked at the **p300 overlap** for selected states.

In [2]:
import pandas as pd

In [3]:
es = pd.read_table('Spectacle/OUTPUTSAMPLE_HG19_NEW/GM12878_20_spectral_segments.bed',header=None)
es.columns = ['chr','lower','upper','state']
es.head()

,chr,lower,upper,state
0,chr10,0,119800,E18
1,chr10,119800,120400,E11
2,chr10,120400,122200,E18
3,chr10,122200,122800,E11
4,chr10,122800,173400,E18


In [21]:
es['chr']=es['chr'].str[3:]

In [136]:
es1 = es[es['state']=='E1'] # Select only rows in state 1

In [137]:
es2 = es[es['state']=='E2'] # Select only rows in state 1

In [138]:
es20 = es[es['state']=='E20'] # Select only rows in state 20

In [7]:
groups_state = dict(list(es.groupby('state')))
groups_state['E20'].head()

,chr,lower,upper,state
13,chr10,179200,179800,E20
156,chr10,958200,958600,E20
298,chr10,1541000,1542000,E20
320,chr10,1698600,1699200,E20
376,chr10,2159800,2160200,E20


In [8]:
p300 = pd.read_table('GM12878_distal_P300_align.hg19.txt.gz', compression = 'gzip', header=None)
p300.columns = ['chr','lower']
p300['upper'] = p300['lower']+200
p300.head()

,chr,lower,upper
0,chr10,100006601,100006801
1,chr10,100011001,100011201
2,chr10,100099401,100099601
3,chr10,100099601,100099801
4,chr10,100099801,100100001


In [15]:
from random import randint, seed
from quicksect import IntervalNode

state_names = ['E1','E2','E3','E4','E5','E6','E12','E13','E14','E20']
for state_name in state_names:
    print state_name
    group_state = groups_state[state_name]
    groups = group_state.groupby('chr')  
    for name, group in groups:
        p = p300[p300['chr']==name]
        query = zip(group['lower'],group['upper'])
        data = zip(p['lower'],p['upper'])

        #Code taken from: https://www.biostars.org/p/99/

        def find(start, end, tree):
            "Returns a list with the overlapping intervals"
            out = []
            tree.intersect( start, end, lambda x: out.append(x) )
            return not not out
            #return True if there is an intersection

        # start the root at the first element
        start, end = data[0]
        tree = IntervalNode( start, end )

        # build an interval tree from the rest of the data
        for start, end in data[1:]:
            tree = tree.insert( start, end )

        overlap = []
        for start, end in query:
            overlap.append(find(start, end , tree))

        group_state.loc[group_state['chr']==name, 'p300'] = map(int,overlap) 
    
    print group_state['p300'].value_counts()
    # Write file
    new_filename = "GM12878_"+state_name+".txt"
    with open(new_filename, 'w') as the_file:
        group_state.to_csv(the_file, sep='\t', index=False)

E1
0    1162
1     755
dtype: int64
E2
1    3297
0    1494
dtype: int64
E3
1    6463
0    6163
dtype: int64
E4
0    9590
1    6158
dtype: int64
E5
0    15880
1     5991
dtype: int64
E6
0    7715
1    1438
dtype: int64
E12
0    27260
1     7002
dtype: int64
E13
0    68777
1     9700
dtype: int64
E14
0    6963
1    1571
dtype: int64
E20
0    4027
1     454
dtype: int64


In [16]:
groups_state['E20'].head()

,chr,lower,upper,state,p300
13,chr10,179200,179800,E20,0
156,chr10,958200,958600,E20,0
298,chr10,1541000,1542000,E20,1
320,chr10,1698600,1699200,E20,0
376,chr10,2159800,2160200,E20,0
